<a href="https://colab.research.google.com/github/MehrdadJalali-AI/GAN-MOF/blob/main/MOFGraph_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Explanation of the MOF Similarity Model**

### **1. Overview**
The MOF (Metal-Organic Framework) similarity model computes pairwise similarities between different MOFs using both molecular and numerical feature-based approaches. The resulting adjacency matrix represents these relationships and is used to construct a similarity graph, where strong connections indicate structural or property-based similarity between MOFs.

---

### **2. Components of the Model**

#### **A. Molecular Similarity (Tanimoto Similarity)**
- Uses **Morgan fingerprints** to encode molecular structures.
- Computes **Tanimoto similarity** for the following molecular components:
  - **Metal SBU (Secondary Building Unit) SMILES**
  - **Linker SMILES**
  - **Metal Cluster SMILES**
  - **Ligand SMILES**
- A weighted sum of these similarities forms the molecular similarity matrix.

#### **B. Feature-Based Similarity (Cosine Similarity)**
- Uses **void fraction, accessible surface area (ASA), accessible volume (AV), pore limiting diameter (PLD), largest cavity diameter (LCD), and largest free path diameter (LFPD)**.
- These numerical features are **normalized using Min-Max scaling** before computing **cosine similarity**.

#### **C. Combined Similarity Matrix**
- The final similarity matrix is obtained by averaging **molecular similarity** and **feature similarity**:
  
  \[ \text{Final Similarity} = \frac{\text{Molecular Similarity} + \text{Feature Similarity}}{2} \]

---

### **3. Adjacency Matrix Construction**
- The **adjacency matrix** is created using the computed similarity values.
- A **threshold** is applied, and values below this threshold are set to **zero** (removing weak connections).
- The matrix is **symmetric**, ensuring that similarity(A, B) = similarity(B, A).
- The diagonal elements are set to **1** (each MOF is identical to itself).

---

### **4. Graph Construction**
- A **graph** is created where:
  - **Nodes** represent MOFs.
  - **Edges** represent strong similarity connections.
  - **Edge weights** correspond to similarity scores.
- Only edges with a similarity above the **threshold** are included.
- A **spring layout** is used to visualize clusters of highly similar MOFs.

---

### **5. Sparsity Calculation**
- The sparsity of the adjacency matrix is calculated as:
  
  \[ \text{Sparsity} = \left(1 - \frac{\text{Number of Edges}}{\text{Total Possible Edges}}\right) \times 100\% \]
  
- Displays the:
  - **Number of MOFs (Nodes)**
  - **Threshold applied**
  - **Number of strong similarity links (Edges)**
  - **Sparsity percentage**

---

### **6. Output and Visualization**
- **MOF_Adjacency_Matrix.csv**: Saves the final similarity matrix.
- **Graph visualization**: Displays the network of MOFs with only strong similarity connections.

---

### **7. Applications**
- **Material Discovery**: Identifying structurally similar MOFs for specific applications.
- **Data-Driven Screening**: Efficiently selecting candidate MOFs for further experimental validation.
- **Clustering & Classification**: Grouping MOFs based on chemical and structural similarities.

---

This model provides an effective way to analyze and visualize MOF relationships, balancing both molecular and feature-based similarities.



In [19]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import warnings
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed  # Parallel processing

# Suppress warnings
warnings.filterwarnings("ignore")

# User-defined parameters
num_mofs = 1000   # Choose the number of MOFs to analyze
threshold = 0.5  # Set the minimum similarity for connections

# Load dataset
df = pd.read_csv("MOF.csv").head(num_mofs)

# Function to compute Morgan fingerprints
def get_fingerprint(smile):
    if pd.isna(smile) or smile == "":
        return None
    mol = Chem.MolFromSmiles(str(smile))
    if mol:
        return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    return None

# Compute fingerprints in parallel for efficiency
df["metal_fp"] = Parallel(n_jobs=-1)(delayed(get_fingerprint)(smile) for smile in df["metal_sbu_smile"])
df["linker_fp"] = Parallel(n_jobs=-1)(delayed(get_fingerprint)(smile) for smile in df["linker_smile"])
df["metal_cluster_fp"] = Parallel(n_jobs=-1)(delayed(get_fingerprint)(smile) for smile in df["metal_cluster_smile"])
df["ligand_fp"] = Parallel(n_jobs=-1)(delayed(get_fingerprint)(smile) for smile in df["ligand_smile"])

# Initialize adjacency matrix
adj_matrix = np.zeros((num_mofs, num_mofs))

# Compute molecular similarity (Tanimoto) using the upper triangle
for i in range(num_mofs):
    for j in range(i + 1, num_mofs):
        metal_sim = linker_sim = cluster_sim = ligand_sim = 0

        if df["metal_fp"][i] and df["metal_fp"][j]:
            metal_sim = DataStructs.TanimotoSimilarity(df["metal_fp"][i], df["metal_fp"][j])

        if df["linker_fp"][i] and df["linker_fp"][j]:
            linker_sim = DataStructs.TanimotoSimilarity(df["linker_fp"][i], df["linker_fp"][j])

        if df["metal_cluster_fp"][i] and df["metal_cluster_fp"][j]:
            cluster_sim = DataStructs.TanimotoSimilarity(df["metal_cluster_fp"][i], df["metal_cluster_fp"][j])

        if df["ligand_fp"][i] and df["ligand_fp"][j]:
            ligand_sim = DataStructs.TanimotoSimilarity(df["ligand_fp"][i], df["ligand_fp"][j])

        # Weighted similarity combination (adjustable weights)
        similarity = (0.3 * metal_sim) + (0.3 * linker_sim) + (0.2 * cluster_sim) + (0.2 * ligand_sim)

        # Fill both upper and lower triangle
        adj_matrix[i, j] = similarity
        adj_matrix[j, i] = similarity

# Feature-based similarity using void fraction and geometric properties
feature_cols = ["void_fraction", "asa (A^2)", "av (A^3)", "pld (A)", "lcd (A)", "lfpd (A)"]
features = df[feature_cols].fillna(0)

# ✅ Normalize numeric features before cosine similarity
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

# Compute feature similarity (Cosine similarity)
feature_similarity = cosine_similarity(features_normalized)

# ✅ Combine molecular and feature similarity (equal weight for both)
adj_matrix = (adj_matrix + feature_similarity) / 2

# ✅ Apply threshold: Set values below threshold to exactly 0
adj_matrix[adj_matrix < threshold] = 0

# ✅ Ensure diagonal values are 1 (MOF identical to itself)
np.fill_diagonal(adj_matrix, 1)

# Convert to DataFrame and save
adj_df = pd.DataFrame(adj_matrix, index=df["Refcode"], columns=df["Refcode"])
adj_df.to_csv("MOF_Adjacency_Matrix.csv")

# --- Graph Visualization ---
G = nx.Graph()

# Add nodes
for refcode in df["Refcode"]:
    G.add_node(refcode)

# Add edges **only for values above threshold**
num_edges = 0
for i in range(num_mofs):
    for j in range(i + 1, num_mofs):
        if adj_matrix[i, j] > 0:  # Now it only adds connections above threshold
            G.add_edge(df["Refcode"][i], df["Refcode"][j], weight=adj_matrix[i, j])
            num_edges += 1

# ✅ Calculate sparsity percentage
total_possible_edges = (num_mofs * (num_mofs - 1)) / 2  # Upper triangle of the adjacency matrix
sparsity_percentage = (1 - (num_edges / total_possible_edges)) * 100

# ✅ Display results
print(f"Number of MOFs (Nodes): {num_mofs}")
print(f"Threshold: {threshold}")
print(f"Number of Links (Edges): {num_edges}")
print(f"Sparsity Percentage: {sparsity_percentage:.2f}%")




Number of MOFs (Nodes): 1000
Threshold: 0.5
Number of Links (Edges): 186141
Sparsity Percentage: 62.73%


# Dataset MOF
This dataset appears to describe a collection of metal-organic frameworks (MOFs) or coordination complexes, likely focusing on their structural and physicochemical properties. Below is an explanation of each column:

**Refcode** – A unique identifier for each structure (e.g., a reference from a crystallographic database like the Cambridge Structural Database).

**metals** – The metal element(s) present in the structure.

**max_metal_coordination_n** – The maximum coordination number of the metal, indicating how many ligands or atoms are directly bonded to it.

**metal_sbu_smile** – SMILES (Simplified Molecular Input Line Entry System) representation of the metal secondary building unit (SBU), which describes its atomic connectivity.

**linker_smile** – The SMILES representation of the organic linker connecting the metal SBUs.

**n_sbu_point_of_extension** – The number of extension points from the metal SBU, representing its connectivity.

**n_linker_point_of_extension** – The number of points where the linker extends and connects to the structure.

**metal_cluster_smile** – SMILES representation of the full metal cluster, which might be more complex than the individual SBU.

**ligand_smile** – SMILES representation of the coordinating ligand(s) attached to the metal.

**n_channel** – The number of channels in the structure, which may relate to porosity or transport properties.

**void_fraction** – The fraction of the unit cell volume that is void or empty space, useful for assessing porosity.

**asa (A²)** – Accessible Surface Area in square angstroms, indicating the exposed surface area within the MOF.

**av (A³)** – Accessible Volume in cubic angstroms, showing the total available volume within the framework.

**pld (A)** – Pore limiting diameter (in angstroms), which is the smallest free diameter for guest molecules to pass through.

**lcd (A)** – Largest cavity diameter, indicating the maximum open space in the framework.

**lfpd (A)** – Largest free path diameter, measuring the longest continuous open pathway within the structure.



In [4]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')
# Change working path
os.chdir('/content/drive/MyDrive/Research/MOF/GAN-NodeGeneration/')
!pip install rdkit

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 48.3 MB/s eta 0:00:00
